In [1]:
from nemo.collections.tts.models import MagpieTTSDecoderModel
from nemo.collections.tts.data.text_to_speech_dataset import MagpieTTSDataset, DatasetSample
from omegaconf.omegaconf import OmegaConf, open_dict
import torch
import os
import soundfile as sf
from IPython.display import display, Audio
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

[NeMo W 2025-06-04 08:08:03 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
      from .autonotebook import tqdm as notebook_tqdm
    
[NeMo W 2025-06-04 08:08:08 nemo_logging:405] Please use the EncDecSpeakerLabelModel instead of this model. EncDecClassificationModel model is kept for backward compatibility with older models.


### Checkpoint Paths

In [2]:
hparams_file = "/datap/misc/Experiments/SimpleT5Explore/LocalTraining_LRH2/Binarize/Magpie-TTS-EN/0/hparams.yaml"
checkpoint_file = "/datap/misc/Experiments/SimpleT5Explore/LocalTraining_LRH2/Binarize/Magpie-TTS-EN/0/checkpoints/epoch17.ckpt"
# codecmodel_path = "/datap/misc/checkpoints/AudioCodec_21Hz_no_eliz.nemo"
codecmodel_path = "/datap/misc/checkpoints/AudioCodec_21Hz_no_eliz.nemo"


# Temp out dir for saving audios
out_dir = "/datap/misc/t5tts_inference_notebook_samples"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

### Load Model

In [3]:
model_cfg = OmegaConf.load(hparams_file).cfg

with open_dict(model_cfg):
    model_cfg.codecmodel_path = codecmodel_path
    model_cfg.train_ds = None
    model_cfg.validation_ds = None


model = MagpieTTSDecoderModel(cfg=model_cfg)
print("Loading weights from checkpoint")
ckpt = torch.load(checkpoint_file, weights_only=False)
model.load_state_dict(ckpt['state_dict'])
print("Loaded weights.")

model.cuda()
model.eval()

[NeMo W 2025-06-04 08:08:10 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      dataset_type: tarred_vocoder
      dataset_args:
        dataset_meta:
          mls_english:
            manifest_path: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/mls_english/filtered_22khz/tarred_audio/train_manifest.json
            tar_filepath: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/mls_english/filtered_22khz/tarred_audio/audio_{0..1999}.tar
          cv:
            manifest_path: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/commonvoice13/22khz/tarred_audio/train_manifest.json
            tar_filepath: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/commonvoice13/22khz/tarred_audio/audio_{0..279}.tar

[NeMo I 2025-06-04 08:08:11 nemo_logging:393] Vector quantizer does not support commit loss.
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:12 nemo_logging:393] PADDING: 1
[NeMo I 2025-06-04 08:08:13 nemo_logging:393] Model AudioCodecModel was successfully restored from /datap/misc/checkpoints/AudioCodec_21Hz_no_eliz.nemo.
[NeMo I 2025-06-04 08:08:28 nemo_logging:393] Local transformer type: autoregressive
Loading weights from checkpoint
Loaded weights.


MagpieTTSDecoderModel(
  (_codec_model): AudioCodecModel(
    (audio_encoder): HiFiGANEncoder(
      (pre_conv): Conv1dNorm(
        (conv): ParametrizedConv1d(
          1, 48, kernel_size=(7,), stride=(1,), padding=(3,), padding_mode=reflect
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): _WeightNorm()
            )
          )
        )
        (activation): Identity()
      )
      (activations): ModuleList(
        (0-4): 5 x CodecActivation(
          (activation): LeakyReLU(negative_slope=0.01)
        )
      )
      (down_sample_conv_layers): ModuleList(
        (0): Conv1dNorm(
          (conv): ParametrizedConv1d(
            48, 96, kernel_size=(4,), stride=(2,), padding=(1,), padding_mode=reflect
            (parametrizations): ModuleDict(
              (weight): ParametrizationList(
                (0): _WeightNorm()
              )
            )
          )
          (activation): Identity()
        )
        (1): C

### Initialize Dataset class and helper functions

In [4]:
test_dataset = MagpieTTSDataset(
    dataset_meta={},
    sample_rate=model_cfg.sample_rate,
    min_duration=0.5,
    max_duration=20,
    codec_model_samples_per_frame=model.codec_model_samples_per_frame,
    bos_id=None,
    eos_id=model.eos_id,
    context_audio_bos_id=model.context_audio_bos_id,
    context_audio_eos_id=model.context_audio_eos_id,
    audio_bos_id=model.audio_bos_id,
    audio_eos_id=model.audio_eos_id,
    num_audio_codebooks=model.num_audio_codebooks,
    prior_scaling_factor=None,
    load_cached_codes_if_available=True,
    dataset_type='test',
    tokenizer_config=None,
    load_16khz_audio=False,
    use_text_conditioning_tokenizer=True,
    pad_context_text_to_max_duration=model.pad_context_text_to_max_duration,
    context_duration_min=model.cfg.get('context_duration_min', 5.0),
    context_duration_max=model.cfg.get('context_duration_max', 5.0),
)
test_dataset.text_tokenizer, test_dataset.text_conditioning_tokenizer = model.tokenizer, model.tokenizer.first_tokenizer



def get_audio_duration(file_path):
    with sf.SoundFile(file_path) as audio_file:
        # Calculate the duration
        duration = len(audio_file) / audio_file.samplerate
        return duration

def create_record(text, context_audio_filepath=None, context_text=None, ):
    dummy_audio_fp = os.path.join(out_dir, "dummy_audio.wav")
    dummy_audio = sf.write(dummy_audio_fp, np.zeros(22050 * 3), 22050)  # 3 seconds of silence
    record = {
        'audio_filepath' : "/datap/misc/LibriTTSfromNemo/LibriTTS/dev-clean/2428/83705/2428_83705_000002_000000.wav",
        'duration': 6.23,
        'text': text,
        'speaker': "dummy",
    }
    if context_text is not None:
        assert context_audio_filepath is None
        record['context_text'] = context_text
    else:
        assert context_audio_filepath is not None
        record['context_audio_filepath'] = context_audio_filepath
        record['context_audio_duration'] = get_audio_duration(context_audio_filepath)
    
    return record

### Set transcript and context pairs to test

In [5]:
import pprint
# Change sample text and prompt audio/text here
audio_base_dir = "/"
test_entries = [
    create_record(
        text="Not that I have a word to say against Mary Ann, except that I never knew a girl with quite so many relations.",
#         text="This is a simple sentence to check my text to speech synthesis model.",
#         text=" How? ",
#         context_text="Speaker and Emotion: | Language:en Dataset:Riva Speaker:Lindy_WIZWIKI |",
#         Lindy_CMU_FEARFUL
#         Lindy_WIZWIKI
#         context_audio_filepath="/datap/misc/LibriTTSfromNemo/LibriTTS/test-clean/7729/102255/7729_102255_000012_000001.wav", # Supply either context_audio_filepath or context_text, not both
        context_audio_filepath="/datap/misc/LibriTTSfromNemo/LibriTTS/dev-clean/2428/83705/2428_83705_000002_000000.wav",
    ),
]

data_samples = []
for entry in test_entries:
    dataset_sample = DatasetSample(
        dataset_name="sample",
        manifest_entry=entry,
        audio_dir=audio_base_dir,
        feature_dir=audio_base_dir,
        text=entry['text'],
        speaker=None,
        speaker_index=0,
        tokenizer_names=["english_phoneme"], # Change this for multilingual: "english_phoneme", "spanish_phoneme", "english_chartokenizer", "german_chartokenizer".. 
    )
    data_samples.append(dataset_sample)
    
test_dataset.data_samples = data_samples

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    collate_fn=test_dataset.collate_fn,
    num_workers=0,
    shuffle=False
)

### Generate With Prior

In [8]:
import matplotlib.pyplot as plt

item_idx = 0
for bidx, batch in enumerate(test_data_loader):
    batch_cuda ={}
    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch_cuda[key] = batch[key].cuda()
        else:
            batch_cuda[key] = batch[key]
    # process_batch_out = model.process_batch(batch_cuda)
    predicted_audio, predicted_audio_lens,_,_ = model.infer_batch(batch_cuda,temperature=0.001,max_decoder_steps=500, autoregressive_after=1)
    for idx in range(predicted_audio.size(0)):
        predicted_audio_np = predicted_audio[idx].float().detach().cpu().numpy()
        predicted_audio_np = predicted_audio_np[:predicted_audio_lens[idx]]
        audio_path = os.path.join(out_dir, f"predicted_audio_{item_idx}.wav")
        sf.write(audio_path, predicted_audio_np, model.cfg.sample_rate)
        print(test_entries[bidx]['text'])
        display(Audio(audio_path))
    
#     print("Processing batch {} out of {}".format(bidx, len(test_data_loader)))
#     model.t5_decoder.reset_cache(use_cache=True)
#     batch_cuda ={}
#     for key in batch:
#         if isinstance(batch[key], torch.Tensor):
#             batch_cuda[key] = batch[key].cuda()
#         else:
#             batch_cuda[key] = batch[key]
#     import time
#     st = time.time()
    
#     for _ in range(1):
#         for use_local_transformer_for_inference in [False, True]:
#             for apply_prior in [False]:
#                 predicted_audio, predicted_audio_lens, _, _, rtf_metrics, cross_attn_np, all_heads_attn_np = model.infer_batch(
#                     batch_cuda, 
#                     max_decoder_steps=430, 
#                     temperature=0.6, 
#                     topk=80, 
#                     use_cfg=False,
#                     cfg_scale=2.5,
#                     prior_epsilon=0.1,
#                     lookahead_window_size=5,
#                     return_cross_attn_probs=True,
#                     estimate_alignment_from_layers=[5],
#                     apply_attention_prior=apply_prior,
#                     apply_prior_to_layers=[0,1,2,3,4,5,6,7,8,9,10,11],
#                     compute_all_heads_attn_maps=True,
#                     start_prior_after_n_audio_steps=0,
#                     use_local_transformer_for_inference=use_local_transformer_for_inference
#                 )
#                 print("generation time", time.time() - st)
#                 pprint.pprint(rtf_metrics)

#                     item_idx += 1
#                     plt.imshow(cross_attn_np[idx])
#                     plt.show()
# #                     for hidx, head_cross_attn in enumerate(all_heads_attn_np[idx]):
# #                         layer_num = hidx // model.cfg.t5_decoder.xa_n_heads
# #                         head_num = hidx % model.cfg.t5_decoder.xa_n_heads
# #                         print("item, layer, head", idx, layer_num, head_num)
# #                         plt.imshow(all_heads_attn_np[idx][hidx])
# #                         plt.show()

#                 print("------------------------------------")
#                 print("------------------------------------")

Timestep: 0
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 1
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 2
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 3
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 4
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 5
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 6
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 7
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 8
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 9
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 10
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 11
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 12
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 13
audio_codes_next.unsqueeze(2).shape torch.Size([1, 8, 1])
Timestep: 14
aud

In [7]:
process_batch_out.keys()

NameError: name 'process_batch_out' is not defined

In [ ]:
pred_audio_codes = model.logits_to_audio_codes(process_batch_out['logits'], process_batch_out['audio_codes_lens_target'])

In [ ]:
pred_audio, pred_audio_lens = model.codes_to_audio(pred_audio_codes, process_batch_out['audio_codes_lens_target'])
# pred_audio, pred_audio_lens = model.codes_to_audio(process_batch_out['audio_codes_target'], process_batch_out['audio_codes_lens_target'])

In [ ]:
predicted_audio_np = pred_audio[0].float().detach().cpu().numpy()
predicted_audio_np = predicted_audio_np[:pred_audio_lens[idx]]
audio_path = os.path.join(out_dir, "t.wav")
sf.write(audio_path, predicted_audio_np, model.cfg.sample_rate)
print(test_entries[bidx]['text'])
display(Audio(audio_path))

In [ ]:
predicted_audio_np.shape

In [ ]:
predicted_audio.shape

In [ ]:
batch

In [ ]:
predicted_audio_np.shape

In [ ]:
pred_audio_codes.shape

In [ ]:
process_batch_out['audio_codes_target'].shape

In [ ]:
batch['audio']